퍼셉트론은 인공 신경망의 가장 기본적인 단위로, 생물학적 뉴런을 수학적으로 모델링한 구조이다. 퍼셉트론은 여러 개의 입력값 x에 각각 가중치 w를 곱하고, 편향 b를 가중합 한 값 z를 계산한다. 이 z값을 계단 함수에 통과시키면 0 혹은 1이라는 이진 출력을 만들어낸다. 퍼셉트론은 신형 분리 문제는 해결할 수 있으나, XOR 문제와 같은 비선형 문제는 해결하지 못하는 한계가 있다.  

이러한 한계를 극복하기 위해 여러 퍼셉트론을 하나의 층으로 형성하고 여러 개의 층으로 쌓아 구성한 것이 신경망이다. 신경망은 입력층, 은닉층, 출력층으로 구성되며, 각 층의 노드들이 다음 층의 노드들과 연결되어 정보를 전달한다. 은닉층에서는 각 노드가 선형 결합된 값을 비선형 활성화 함수를 통해 처리함으로써, 입력과 출력 사이의 복잡한 관계를 학습할 수 있게 된다.  

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

In [2]:
# 1. 데이터 로딩 및 전처리
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

train_dataset = datasets.MNIST(root='./data', train=True, transform=transform, download=True)
test_dataset  = datasets.MNIST(root='./data', train=False, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True) # 훈련 데이터 로더, 배치 사이즈 64, 셔플 True
test_loader  = DataLoader(test_dataset, batch_size=64, shuffle=False) # 테스트 데이터 로더, 배치 사이즈 64, 셔플 False

Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 9912422/9912422 [00:02<00:00, 3686756.14it/s]


Extracting ./data\MNIST\raw\train-images-idx3-ubyte.gz to ./data\MNIST\raw

Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 28881/28881 [00:00<00:00, 132721.05it/s]


Extracting ./data\MNIST\raw\train-labels-idx1-ubyte.gz to ./data\MNIST\raw

Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 1648877/1648877 [00:02<00:00, 797768.73it/s] 


Extracting ./data\MNIST\raw\t10k-images-idx3-ubyte.gz to ./data\MNIST\raw

Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 4542/4542 [00:00<00:00, 3699131.80it/s]

Extracting ./data\MNIST\raw\t10k-labels-idx1-ubyte.gz to ./data\MNIST\raw



In [3]:
# 2. MLP 모델 정의
class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(28 * 28, 256),   # 입력층 → 첫 은닉층, 이미지 크기 28x28x1 을 256 차원으로 변환
            nn.ReLU(),                 # 활성화 함수
            nn.Linear(256, 128),       # 첫 은닉층 → 두 번째 은닉층     256 차원에서 128 차원으로 변환
            nn.ReLU(),
            nn.Linear(128, 10)         # 두 번째 은닉층 → 출력층 (10 클래스) 128 차원에서 10 차원으로 변환, 10개의 클래스 (0~9)
        )

    def forward(self, x):
        x = x.view(-1, 28 * 28)  # 이미지를 1D 벡터로 펼치기
        return self.model(x)

# 3. 학습 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = MLP().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001) # Adam 옵티마이저 사용, 학습률 0.001

# 4. 학습 루프
for epoch in range(5):  # 에폭 수 조정 가능
    model.train()
    total_loss = 0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()             # 기울기 초기화
        outputs = model(images)           # 모델에 이미지 입력, outputs은 모델의 예측값
        loss = criterion(outputs, labels) # 손실 함수 계산, 예측값과 실제 레이블 비교
        loss.backward()                   # 역전파 수행, 기울기 계산
        optimizer.step()                  # 계산한 기울기를 가지고 가중치 업데이트
        
        total_loss += loss.item()         # 손실값 누적
    
    print(f'Epoch {epoch+1}, Loss: {total_loss:.4f}')

Epoch 1, Loss: 314.8791
Epoch 2, Loss: 143.6769
Epoch 3, Loss: 103.3053
Epoch 4, Loss: 86.9063
Epoch 5, Loss: 70.1854


In [4]:
# 5. 테스트 정확도 측정
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Test Accuracy: {100 * correct / total:.2f}%')

Test Accuracy: 97.37%
